In [1]:
import warnings
warnings.filterwarnings('ignore')
import keras
import sys 

from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, BatchNormalization

Using TensorFlow backend.


In [24]:
datagen = ImageDataGenerator(rotation_range=30, 
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True
                            )

if 'win32' in sys.platform:
    data_directory='F:\\Segmentation_Data\\'
    full_training_directory = data_directory + 'Labelled_imgs\\data_for_generator\\training\\'
    full_validation_directory = data_directory +'Labelled_imgs\\data_for_generator\\validation\\'

In [25]:
train_generator = datagen.flow_from_directory(full_training_directory, target_size=(128, 128), color_mode="rgb")
validation_generator = datagen.flow_from_directory(full_validation_directory, target_size=(128, 128), color_mode='rgb')
num_classes = len(train_generator.class_indices)

Found 19890 images belonging to 7 classes.
Found 8524 images belonging to 7 classes.


In [26]:
model = Sequential()
model.add(Conv2D(20, (5,5), activation='relu', input_shape=(128,128,3)))
model.add(BatchNormalization(momentum=0.9))
model.add(MaxPooling2D())
model.add(Conv2D(50, (5,5), activation='relu'))
model.add(BatchNormalization(momentum=0.9))
model.add(MaxPooling2D())
model.add(Conv2D(100, (4,4), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(200, (4,4), activation='relu'))
model.add(MaxPooling2D())
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(500))
model.add(Dropout(rate=0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [27]:
for layer in model.layers:
    print(layer.output_shape)

(None, 124, 124, 20)
(None, 124, 124, 20)
(None, 62, 62, 20)
(None, 58, 58, 50)
(None, 58, 58, 50)
(None, 29, 29, 50)
(None, 26, 26, 100)
(None, 13, 13, 100)
(None, 10, 10, 200)
(None, 5, 5, 200)
(None, 5, 5, 200)
(None, 5000)
(None, 500)
(None, 500)
(None, 7)
(None, 7)


In [28]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator,
                    verbose=1, 
                    steps_per_epoch=1000, 
                    epochs=1, 
                    validation_data=validation_generator,
                   validation_steps=300)

Epoch 1/1
1000/1000 [==============================] - 160s 160ms/step - loss: 1.1389 - acc: 0.5834 - val_loss: 0.9660 - val_acc: 0.6377
